In [13]:
import pandas as pd
from typing import Any, Callable
from tabulate import tabulate

class DT:
    @staticmethod
    def is_4digits(value: Any) -> bool:
        return isinstance(value, str) and len(value) == 4 and value.isdigit()

    @staticmethod
    def is_5digits(value: Any) -> bool:
        return isinstance(value, str) and len(value) == 5 and value.isdigit()

    @staticmethod
    def is_empty(value: Any) -> bool:
        return pd.isna(value) or (isinstance(value, str) and value.strip() == '')

    @staticmethod
    def is_not_empty(value: Any) -> bool:
        return not DT.is_empty(value)

class EditorFactory:
    def __init__(self, decision_table: pd.DataFrame):
        self.decision_table = decision_table
        self.dt_functions = {func: getattr(DT, func) for func in dir(DT) 
                             if callable(getattr(DT, func)) and not func.startswith("__")}
        print(f"Initialized dt_functions: {self.dt_functions}")

    def check_condition(self, value: Any, condition: Any) -> bool:
        print(f"Checking condition: value={value}, condition={condition}")
        
        if pd.isna(condition) or condition == "any":
            return True
        
        if isinstance(condition, str):
            if condition in self.dt_functions:
                result = self.dt_functions[condition](value)
                print(f"DT function {condition} result: {result}")
                return result
            
            # OR条件の処理
            if "," in condition:
                conditions = [c.strip() for c in condition.split(',')]
                return any(self.check_condition(value, c) for c in conditions)
            
            # 文字列の比較
            return str(value) == condition
        
        # その他の型の比較
        return value == condition

    def evaluate_conditions(self, row: pd.Series) -> str:
        print(f"Evaluating conditions for row: {row}")
        for idx, decision_row in self.decision_table.iterrows():
            print(f"Checking decision row {idx}: {decision_row}")
            conditions_met = [self.check_condition(row[col], decision_row[col]) for col in row.index]
            print(f"Conditions met: {conditions_met}")
            if all(conditions_met):
                result = decision_row['DecisionResult']
                print(f"All conditions met, returning DecisionResult: {result}")
                return result
        print("No matching conditions found, returning DataFrameEditorDefault")
        return 'DataFrameEditorDefault'

    def create_editor(self, row: pd.Series) -> str:
        return self.evaluate_conditions(row)

# テスト用のコード
decision_table = pd.DataFrame({
    'DecisionResult': ['DataFrameEditor1', 'DataFrameEditor2', 'DataFrameEditor3', 'DataFrameEditor4', 'DataFrameEditor5',
                       'DataFrameEditor6', 'DataFrameEditor7', 'DataFrameEditor8', 'DataFrameEditor9', 'DataFrameEditor10'],
    'column1': [1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
    'column2': [1, 0, 1, 1, 1, 1, 1, 'is_not_empty', 0, 1],
    'column3': [1, 'is_4digits', 0, 1, 1, 1, 1, 1, 1, 0],
    'column4': [1, 1, 'is_5digits', 0, 1, 1, 'any', 1, 1, 1],
    'column5': [1, 1, 1, 'any', 0, 'any', 1, 1, 1, 1],
    'column6': [1, 1, 2, 1, '1,2,3', 0, 1, 1, 'is_empty', 1],
    'column7': [1, 1, 1, 1, 1, 1, 0, 1, 1, 1]
})

decision_table = decision_table.astype(object)
print(decision_table.dtypes)

factory = EditorFactory(decision_table)

# テストデータ
test_data = pd.DataFrame({
    'column1': [1, 1, 1, 1, 1, 0],
    'column2': [1, 1, 0, 1, 'non-empty', 0],
    'column3': [1, 1, '1234', 0, 1, 1],
    'column4': [1, 1, 1, '12345', 0, 1],
    'column5': [1, 0, 1, 1, 3, 1],
    'column6': [1, 2, 1, 2, 1, ''],
    'column7': [1, 1, 1, 1, 1, 1],
})
test_data = test_data.astype(object)
print(test_data.dtypes)

print(tabulate(decision_table, headers=decision_table.columns))
print(tabulate(test_data, headers=test_data.columns))

for _, row in test_data.iterrows():
    result = factory.create_editor(row)
    print(f"Selected editor for row {row}: {result}")
    print("-" * 50)

DecisionResult    object
column1           object
column2           object
column3           object
column4           object
column5           object
column6           object
column7           object
dtype: object
Initialized dt_functions: {'is_4digits': <function DT.is_4digits at 0x7f2193d35ee0>, 'is_5digits': <function DT.is_5digits at 0x7f2193d35da0>, 'is_empty': <function DT.is_empty at 0x7f2193d36340>, 'is_not_empty': <function DT.is_not_empty at 0x7f2193d360c0>}
column1    object
column2    object
column3    object
column4    object
column5    object
column6    object
column7    object
dtype: object
    DecisionResult       column1  column2       column3     column4     column5    column6      column7
--  -----------------  ---------  ------------  ----------  ----------  ---------  ---------  ---------
 0  DataFrameEditor1           1  1             1           1           1          1                  1
 1  DataFrameEditor2           1  0             is_4digits  1           1  